In [33]:
import torch
import numpy as np
from torch import nn, optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.utils.data as data


In [34]:
class MyDataset(data.Dataset):
    def __init__(self, x, y):
        self.tx = torch.Tensor(x)
        self.ty = torch.Tensor(y)

    def __getitem__(self, index):#返回的是tensor
        ttx, tty = self.tx[index], self.ty[index]
        return ttx, tty

    def __len__(self):
        return len(self.ty)


# dataset = MyDataset(x, y)
# testdataset = MyDataset(testx, testy)

In [44]:
class FC_Net(nn.Module):
    def __init__(self, in_dim, n_hidden_1, out_dim):

        super(FC_Net, self).__init__()

        self.layer1 = nn.Sequential(nn.Linear(in_dim, n_hidden_1), nn.BatchNorm1d(n_hidden_1), nn.ReLU(True))

        self.layer2 = nn.Sequential(nn.Linear(n_hidden_1, out_dim))



    def forward(self, x):
#        print(x.shape)
        x = self.layer1(x)
#        print(x.shape)
        x = self.layer2(x)
#        print(x.shape)

        return x

In [43]:
train_x=[]
train_y=[]
with open("./dataset/train_ready") as f:
    for l in f.readlines():
        l = l.strip("\n")
        tmp = l.split('|')
        train_y.append(int(tmp[1]))
        vec = [-float(i[1:]) if i[0]=='-'else float(i) for i in tmp[0][1:-1].split(",")]
        assert len(vec)==1024
        train_x.append(vec)
train_x = np.array(train_x)
train_y = np.array(train_y)

dev_x=[]
dev_y=[]
with open("./dataset/dev_ready") as f:
    for l in f.readlines():
        l = l.strip("\n")
        tmp = l.split('|')
        dev_y.append(int(tmp[1]))
        vec = [-float(i[1:]) if i[0]=='-'else float(i) for i in tmp[0][1:-1].split(",")]
        assert len(vec)==1024
        dev_x.append(vec)
dev_x = np.array(dev_x)
dev_y = np.array(dev_y)
# print(len(dev_x))
# print(len(dev_x[0]))
# print(dev_y[0])


dataset = MyDataset(train_x, train_y)
testdataset = MyDataset(dev_x, dev_y)

In [55]:
# 定义一些超参数

batch_size = 400

learning_rate = 0.005

num_epoches = 10





# 数据集的下载器

train_dataset = dataset
test_dataset = testdataset

#test_dataset = datasets.MNIST(root='./data', train=False, transform=data_tf)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


# 选择模型

#model = net.simpleNet(28 * 28, 300, 100, 10)

# model = net.Activation_Net(28 * 28, 300, 100, 10)

model = FC_Net(1024, 512, 2)
model.train()
model.cuda()
# 定义损失函数和优化器

criterion = nn.CrossEntropyLoss()

#optimizer = optim.SGD(model.parameters(), lr=learning_rate)
optimizer = optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.99))
#optimizer = optim.RMSprop(model.parameters(), lr=learning_rate, alpha = 0.9)


# 训练模型

epoch = 0
epoch_arr=[]
loss_arr=[]
for data in train_loader:
    feature, label = data
    #print(img.shape, label.shape)
    label.long()
#    img = img.view(img.size(0), -1)

    if torch.cuda.is_available():

        feature = feature.cuda()

        label = label.cuda()

    else:

        feature = Variable(feature)

        label = Variable(label)

    #print(label.type)
    label = label.long()
    #print(label)
    out = model(feature)

    loss = criterion(out, label)

    print_loss = loss.data.item()
    loss_arr.append(print_loss)

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()

    epoch += 1
    epoch_arr.append(epoch)
    if epoch % num_epoches == 0:

        print('epoch: {}, loss: {:.4}'.format(epoch, loss.data.item()))
#plt.figure()
#plt.plot(epoch_arr, loss_arr)
#plt.xlabel('epoches')
#plt.ylabel('trainning loss')
#plt.show()
print('start testing...............')
# 模型评估

model.eval()
model.cuda()
eval_loss = 0


eval_loss = 0
eval_acc = 0
pt = 0
rt = 0
tp = 0
for data in test_loader:

    feature, label = data
    print(feature.shape)
    # img = img.view(img.size(0), -1)
    label = label.long()
    if torch.cuda.is_available():
        feature = feature.cuda()

        label = label.cuda()

    out = model(feature)

    loss = criterion(out, label)

    eval_loss += loss.data.item() * label.size(0)

    _, pred = torch.max(out, 1)

    num_c = (pred == label)
    num_correct = (pred == label).sum()
    pt += pred.sum().item()
    rt += label.sum().item()
    tp += (num_c.long()*pred.long()).sum().item()
    eval_acc += num_correct.item()

print('Test Loss: {:.6f}, Acc: {:.6f}'.format(

    eval_loss / (len(test_dataset)),

    eval_acc / (len(test_dataset))
))
print("tp: "+str(tp))
print("pt: "+str(pt))
print("rt: "+str(rt))
print('precsion is {:.6f} and recall is{:.6f}'.format(tp/pt,tp/rt))
p = tp/pt
r = tp/rt
f = 2*p*r/(p+r)
print("f1 score is {:.6f}".format(f))

print('end---------------------')

epoch: 10, loss: 0.4625
epoch: 20, loss: 0.3575
epoch: 30, loss: 0.3768
epoch: 40, loss: 0.365
epoch: 50, loss: 0.3107
start testing...............
torch.Size([400, 1024])
torch.Size([400, 1024])
torch.Size([400, 1024])
torch.Size([400, 1024])
torch.Size([400, 1024])
torch.Size([400, 1024])
torch.Size([400, 1024])
torch.Size([25, 1024])
Test Loss: 0.368405, Acc: 0.843540
tp: 690
pt: 819
rt: 1003
precsion is 0.842491 and recall is0.687936
f1 score is 0.757409
end---------------------
